In [22]:
import os
import cv2
import dlib
import numpy as np
import torch
import torch.nn as nn
from deepface import DeepFace
import tensorflow.lite as tflite

class CombinedFaceModel(nn.Module):
    def __init__(self):
        super(CombinedFaceModel, self).__init__()
        self.landmark_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
        self.interpreter = tflite.Interpreter("ssd_mobilenet_v2_face_quant_postprocess.tflite")
        self.interpreter.allocate_tensors()

    def forward(self, img):
        # Convert PyTorch tensor to NumPy if necessary
        if isinstance(img, torch.Tensor):
            numpy_img = img.permute(0, 2, 3, 1).detach().cpu().numpy().copy()
            if numpy_img.ndim == 4:
                numpy_img = numpy_img[0]

        # Resize and run TFLite inference
        numpy_img_resized = cv2.resize(numpy_img, (320, 320), interpolation=cv2.INTER_AREA)
        faces = self.run_tflite_inference(numpy_img_resized)

        embeddings = []
        for face in faces:
            routesCrd, landmarksTuples, face_img = self.faceAlignment(numpy_img, face)
            rotated_face_img = self.faceRotation(routesCrd, landmarksTuples, face_img)
            embedding = self.faceRepresentation(rotated_face_img)
            embeddings.append(embedding)

        return embeddings

    def run_tflite_inference(self, img):
        # Prepare the input tensor
        tensor = self.input_tensor()
        
        # 명시적으로 데이터를 복사하여 참조를 피합니다
        tensor[...] = img.astype(np.float32).copy()  # Use .copy() to avoid referencing issues

        # Run the inference
        self.interpreter.invoke()

        # Get outputs
        return self.get_output(0.5, (img.shape[1] / 320, img.shape[0] / 320))

    def input_tensor(self):
        """Returns input tensor view as numpy array."""
        tensor_index = self.interpreter.get_input_details()[0]['index']
        return self.interpreter.tensor(tensor_index)()[0]

    def get_output(self, score_threshold, image_scale):
        boxes = self.output_tensor(0)
        class_ids = self.output_tensor(1)
        scores = self.output_tensor(2)
        count = int(self.output_tensor(3))

        width, height = 320, 320  # Resized image size
        image_scale_x, image_scale_y = image_scale
        sx, sy = width / image_scale_x, height / image_scale_y

        def make(i):
            ymin, xmin, ymax, xmax = boxes[i]
            return {
                'id': int(class_ids[i]),
                'score': float(scores[i]),
                'bbox': {
                    'xmin': int(xmin * sx),
                    'ymin': int(ymin * sy),
                    'xmax': int(xmax * sx),
                    'ymax': int(ymax * sy)
                }
            }

        return [make(i) for i in range(count) if scores[i] >= score_threshold]

    def output_tensor(self, i):
        tensor = self.interpreter.tensor(self.interpreter.get_output_details()[i]['index'])()
        return np.squeeze(tensor)

    def faceAlignment(self, img, face):
        baseImg = img.copy()
        dlib_box = dlib.rectangle(int(face['bbox']['xmin']), int(face['bbox']['ymin']),
                                  int(face['bbox']['xmax']), int(face['bbox']['ymax']))
        landmarks = self.landmark_detector(img, dlib_box)
        landmarksTuples = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)]

        routes = [i for i in range(16, -1, -1)] + [i for i in range(17, 27)] + [16]
        routesCrd = []
        for i in range(len(routes) - 1):
            sourceCrd = landmarksTuples[routes[i]]
            targetCrd = landmarksTuples[routes[i + 1]]
            routesCrd.append(sourceCrd)

        routesCrd.append(routesCrd[0])
        mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        mask = cv2.fillConvexPoly(mask, np.array(routesCrd, np.int32), 1)
        out = np.zeros_like(img)
        out[mask == 1] = img[mask == 1]
        return routesCrd, landmarksTuples, out

    def faceRotation(self, routesCrd, landmarksTuples, out):
        delta_y = abs(landmarksTuples[44][1] - landmarksTuples[38][1])
        delta_x = abs(landmarksTuples[44][0] - landmarksTuples[38][0])
        angle = np.arctan2(delta_y, delta_x)
        angle_degrees = np.degrees(angle)

        (h, w) = out.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, -angle_degrees, 1.0)
        rotated = cv2.warpAffine(out, M, (w, h))

        x_min, x_max = min(pt[0] for pt in routesCrd), max(pt[0] for pt in routesCrd)
        y_min, y_max = min(pt[1] for pt in routesCrd), max(pt[1] for pt in routesCrd)
        face_img = rotated[y_min:y_max, x_min:x_max]

        return face_img

    def faceRepresentation(self, face_img):
        if face_img is None:
            raise ValueError("Invalid image input - None. Please provide a valid image.")
        try:
            representation = DeepFace.represent(face_img, detector_backend='retinaface', model_name='ArcFace')
            return representation[0]['embedding']
        except Exception as e:
            print(f"Error in faceRepresentation: {e}")
            return None

# 모델 인스턴스 생성
model = CombinedFaceModel()

# 이미지 읽기 및 전처리
verifyFace_path = 'img3.jpg'
image = cv2.imread(verifyFace_path)

# 이미지를 PyTorch 텐서로 변환
image_tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0)  # (H, W, C) -> (N, C, H, W)

# 더미 입력 생성
dummy_face = {
    'bbox': {
        'xmin': 0,
        'ymin': 0,
        'xmax': 320,
        'ymax': 320
    }
}

# ONNX export
torch.onnx.export(
    model,
    (image_tensor, dummy_face),
    "combined_face_model.onnx",
    input_names=['img', 'face'],
    output_names=['embedding'],
    dynamic_axes={'img': {0: 'batch_size'}, 'face': {0: 'batch_size'}},
    operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
    do_constant_folding=True
)


RuntimeError: There is at least 1 reference to internal data
      in the interpreter in the form of a numpy array or slice. Be sure to
      only hold the function returned from tensor() if you are using raw
      data access.